# Documentation

https://praw.readthedocs.io/en/stable/getting_started/authentication.html

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [2]:
import praw

user_agent = "Reddit_Scrapper 1.0 by u/Ansamzi"
reddit = praw.Reddit(
    client_id="GxxMVD29d1RBDXIqdjDnjA",
    client_secret="zTpzmgGGuSCnL2BlzptPX5yEBC_CCw",
    redirect_uri="http://localhost:8080",
    user_agent=user_agent
    )

In [3]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 6, 30)

# Calculate the difference in days
days_difference = (end_date - start_date).days

In [4]:
headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=10):
    print(submission.created_utc)
    print(submission.title)#Subreddit Title
    print(submission.id) #ID
    print(submission.author) #Author of the subreddit
    print(submission.created_utc) #Date and time being created
    print(submission.score) # Average Score
    print(submission.upvote_ratio) # Upvote ratio
    print(submission.url) # Like to the Subreddit
    break

print(len(headlines))

1701489754.0
/r/WorldNews Live Thread: Russian Invasion of Ukraine Day 647, Part 1 (Thread #793)
188v4ex
WorldNewsMods
1701489754.0
670
0.96
https://www.reddit.com/live/18hnzysb1elcs
0


In [5]:
headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=None):
    headlines.add(submission.title)
    headlines.add(submission.created_utc)
print(len(headlines))

1157


In [6]:
headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=None):
    headlines.add(submission.title)
    headlines.add(datetime.utcfromtimestamp(submission.created_utc))

In [7]:
headlines

{datetime.datetime(2023, 11, 29, 15, 39, 55),
 datetime.datetime(2023, 11, 28, 14, 59, 37),
 datetime.datetime(2023, 11, 30, 16, 42, 5),
 datetime.datetime(2023, 11, 29, 17, 21, 25),
 'Somalia arms embargo lifted',
 datetime.datetime(2023, 11, 29, 15, 31, 9),
 'Human Rights Watch says rocket misfire likely cause of deadly Gaza hospital blast',
 datetime.datetime(2023, 11, 29, 12, 17, 45),
 'Poland is encouraging truckers and farmers to block thousands of Ukrainian trucks at the border: With the connivance of the Polish authorities, transporters and agriculturalists have left Ukrainian drivers stuck for days, claiming they represent unfair competition',
 'India in Talks With Boeing, Blue Origin For Space Partnerships',
 datetime.datetime(2023, 12, 2, 11, 45, 49),
 'GMB members vote for strike action at Amazon Minworth site',
 datetime.datetime(2023, 12, 2, 2, 5, 4),
 'Nepal registers first same-sex marriage hailed as win for LGBT rights',
 'South Korean City Turns to Blind Dates to Incr

In [8]:
#Create a dataframe from the scrapped data 
news_df = pd.DataFrame(headlines)
news_df.head()

,0
0,2023-11-29 15:39:55
1,2023-11-28 14:59:37
2,2023-11-30 16:42:05
3,2023-11-29 17:21:25
4,Somalia arms embargo lifted


In [9]:
# Get news from the specified day
start_date = datetime(2023, 11, 1, 0, 0, 0)
end_date = datetime(2023, 12, 1, 0, 0, 0)

# Convert start and end times to Unix timestamp
start_timestamp = start_date.timestamp()
end_timestamp = end_date.timestamp()

headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=10):
   if start_timestamp <= submission.created_utc <= end_timestamp:
       headlines.add(submission.title)

In [10]:
headlines

{'/r/WorldNews Live Thread for 2023 Israel-Hamas Crisis (Thread 42)'}

# Scrapping reddit through google search

Note: scraping reddit through the reddit API was a failure because of the time limit, we only managed to get posts from the last month.

Subreddit to scrap:

* r/worldnews
* r/Wallstreetbets
* r/stocks 
* r/investing
* r/Microsoft


In [1]:
from googlesearch import search
from bs4 import BeautifulSoup
import requests
import time

ImportError: cannot import name 'search' from 'googlesearch' (unknown location)

In [ ]:
def google_search(query, num_results=3, custom_date=None):
    search_results = []

    # Perform the Google search
    if custom_date:
        query += f" after:{custom_date}"
    
    for result_url in search(query, num_results=num_results):
        search_results.append(result_url)

    # Scrape the text from the search results
    result_texts = []
    for result_url in search_results:
        try:
            response = requests.get(result_url)
            soup = BeautifulSoup(response.text, 'html.parser')
            time.sleep(10)
            paragraphs = soup.find_all('p')
            text = ' '.join([p.get_text() for p in paragraphs])
            result_texts.append(text)
        except Exception as e:
            print(f"Error scraping {result_url}: {str(e)}")
            result_texts.append("Error scraping the page")

    return result_texts

In [ ]:
query = 'reddit post r/WorldNews'
custom_date = "2023-02-26"  
num_results = 1

# results = google_search(query, num_results, custom_date)

# for i, text in enumerate(results, start=1):
#     print(f"Result {i}:\n{text}\n{'='*50}\n")

In [ ]:
query += f" after:{custom_date} before:{custom_date}"
query_results = search(query, num_results)

for i, result_url in enumerate(query_results, start=0):
    print(f"Result {i}: {result_url}")

Result 0: https://www.reddit.com/t/donbas/
Result 1: https://www.reddit.com/r/TrueAskReddit/comments/2n2yv9/in_terms_of_pure_military_strength_what_countries/
Result 2: https://www.reddit.com/r/PoliticalCompassMemes/comments/11cpyan/damn_they_were_right_i_guess/


In [ ]:
query_results = search(query, num_results)
for i, result_url in enumerate(query_results, start=1):
    print(result_url.split('/'))

['https:', '', 'www.reddit.com', 't', 'donbas', '']
['https:', '', 'www.reddit.com', 'r', 'TrueAskReddit', 'comments', '2n2yv9', 'in_terms_of_pure_military_strength_what_countries', '']
['https:', '', 'www.reddit.com', 'r', 'PoliticalCompassMemes', 'comments', '11cpyan', 'damn_they_were_right_i_guess', '']


In [ ]:
response = search(query)
for result in response.results:
    print("Title: " + result.title)
    print("Content: " + result.getText())

NameError: name 'search' is not defined